<a href="https://colab.research.google.com/github/NischalSuresh/mentalHealth_chatbot/blob/main/chatbot_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install necessary libraries

In [6]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Import necesssary packages

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset

## Load the dataset Counsel_chat from huggingface (https://github.com/nbertagnolli/counsel-chat)

In [34]:
dataset = load_dataset("nbertagnolli/counsel-chat")
dataset = dataset['train'].train_test_split(test_size=0.1)

Repo card metadata block was not found. Setting CardData to empty.


In [35]:
dataset

DatasetDict({
    train: Dataset({
        features: ['questionID', 'questionTitle', 'questionText', 'questionLink', 'topic', 'therapistInfo', 'therapistURL', 'answerText', 'upvotes', 'views'],
        num_rows: 2497
    })
    test: Dataset({
        features: ['questionID', 'questionTitle', 'questionText', 'questionLink', 'topic', 'therapistInfo', 'therapistURL', 'answerText', 'upvotes', 'views'],
        num_rows: 278
    })
})

In [36]:
len(dataset['train'])

2497

## Prompt template used in Llama-2 original model :
[INST] \<\<SYS\>\>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
\<\</SYS\>\>
{prompt}[/INST] <br>
 Source:
 - https://huggingface.co/TheBloke/Llama-2-70B-Chat-GGML
 - https://github.com/huggingface/blog/blob/main/llama2.md#fine-tuning-with-peft

 Refer chat_completion fn from generation.py for finetuning template - https://github.com/facebookresearch/llama/blob/main/llama/generation.py#L45

## Format the text prompt to required format

In [16]:
def formatting_prompts_func(counsel_data):
  output_texts = []
  for i in range(len(counsel_data)):
    if(counsel_data['answerText'][i]) is None:
      pass
    else:
      question = (counsel_data['questionText'][i]) if (counsel_data['questionText'][i]) is not None else (counsel_data['questionTitle'][i])
      text = f"</s>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.\n### Question: {question.strip()}\n### Answer: {(counsel_data['answerText'][i]).strip()}</s>"
      output_texts.append(text)
  return output_texts

## Check data for error (missing entries)

In [17]:
print(len(dataset['train']))
for i in range(len(dataset['train'])):
  try:
    question = (dataset['train']['questionText'][i]) if (dataset['train']['questionText'][i]) is not None else (dataset['train']['questionTitle'][i])
    answer = (dataset['train']['answerText'][i])
  except:
    print("error",i)


2497


In [18]:
out = formatting_prompts_func(dataset['train'])

## check data output

In [50]:
print(out[1])

</s>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
### Question: I have a fear of something and I want to face that fear to overcome it, but I don't know how. What can I do?
### Answer: Your fear may have deeper roots within your sense of who you are, than you realize.   Fears are sometimes irrational so that logic doesn't get rid of them.Think about whether you felt secure and confident as a child.  Also, did any major bad events happen to you with other people or situations when you were growing up?Often these overwhelming situations of childhood stay with us as fears of situations in our adult lives.  If the root of the problem w the fear is from long ago, then probably a therapist who can ask you questions which help you remember upsetting childhood circumstances, may help you to dissolve the current fear.Another possibility is CBT, cognitive behavior therapy which teaches people short term mantras to do something which

## checking Llama format from llama-recipes

In [ ]:
!pip install llama-recipes


In [ ]:
from llama_recipes.utils.dataset_utils import get_preprocessed_dataset
from llama_recipes.configs.datasets import samsum_dataset

temp_dataset = get_preprocessed_dataset(tokenizer, samsum_dataset, 'train')

In [ ]:
temp_dataset

In [ ]:
tokenizer.decode(temp_dataset[1]['input_ids'])